In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import matplotlib.patches as patches
import cv2
import numpy as np


Informatiile necesare pentru conectarea la Azure computer vision

In [3]:
# Azure credentials
subscription_key = "8EcSSHwbjpywqtIhFz1xM8ymmcoJyDiNNrxdXKkUYsTK9IO72T9XJQQJ99BCACi5YpzXJ3w3AAAFACOGLWpt"
endpoint = "https://teodorvitelaru.cognitiveservices.azure.com/"

# Authenticate client
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))


Pentru clasificarea imaginilor:

In [11]:
ground_truth = {
    "bike1.jpg": "Bicycle",
    "bike02.jpg": "Bicycle",
    "bike03.jpg": "Bicycle",
    "bike04.jpg": "Bicycle",
    "bike05.jpg": "Bicycle",
    "bike06.jpg": "Bicycle",
    "bike07.jpg": "Bicycle",
    "bike08.jpg": "Bicycle",
    "bike09.jpg": "Bicycle",
    "bike10.jpg": "Bicycle",
    "traffic01.jpg": "No Bicycle",
    "traffic02.jpg": "No Bicycle",
    "traffic03.jpg": "No Bicycle",
    "traffic04.jpg": "No Bicycle",
    "traffic05.jpg": "No Bicycle",
    "traffic06.jpg": "No Bicycle",
    "traffic07.jpg": "No Bicycle",
    "traffic08.jpg": "No Bicycle",
    "traffic09.jpg": "No Bicycle",
    "traffic10.jpg": "No Bicycle",
}

# Classify images
def classify_images(image_folder):
    results = {}
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        with open(img_path, "rb") as image_stream:
            analysis = computervision_client.analyze_image_in_stream(image_stream, visual_features=[VisualFeatureTypes.tags])
            tags = [tag.name for tag in analysis.tags]
            results[img_name] = "Bicycle" if "bicycle" in tags else "No Bicycle"
    return results

# Example usage
image_folder = "images/bikes"
predicted_labels = classify_images(image_folder)
print(predicted_labels)

y_true = [ground_truth[img] for img in ground_truth.keys()]
y_pred = [predicted_labels[img] for img in ground_truth.keys()]

#calculam metricile
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, pos_label="Bicycle")
recall = recall_score(y_true, y_pred, pos_label="Bicycle")
f1 = f1_score(y_true, y_pred, pos_label="Bicycle")

# Afișăm rezultatele
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


{'bike02.jpg': 'Bicycle', 'bike03.jpg': 'Bicycle', 'bike04.jpg': 'Bicycle', 'bike05.jpg': 'Bicycle', 'bike06.jpg': 'Bicycle', 'bike07.jpg': 'Bicycle', 'bike08.jpg': 'Bicycle', 'bike09.jpg': 'Bicycle', 'bike1.jpg': 'Bicycle', 'bike10.jpg': 'Bicycle', 'traffic01.jpg': 'No Bicycle', 'traffic02.jpg': 'No Bicycle', 'traffic03.jpg': 'No Bicycle', 'traffic04.jpg': 'No Bicycle', 'traffic05.jpg': 'No Bicycle', 'traffic06.jpg': 'No Bicycle', 'traffic07.jpg': 'No Bicycle', 'traffic08.jpg': 'No Bicycle', 'traffic09.jpg': 'No Bicycle', 'traffic10.jpg': 'No Bicycle'}
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1-Score: 1.00


Este evident, task ul de la b dureaza mai mult deoarece este nevoie sa introduc manual acel
border pentru fiecare imagine

In [6]:
# Detect objects and draw bounding boxes
def detect_and_draw_bicycles(image_folder, output_folder):
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        with open(img_path, "rb") as image_stream:
            # Analizăm imaginea folosind serviciul Computer Vision
            analysis = computervision_client.analyze_image_in_stream(image_stream, visual_features=[VisualFeatureTypes.objects])
            
            # Deschidem imaginea
            img = Image.open(img_path)
            draw = ImageDraw.Draw(img)
            
            # Desenăm chenarul roșu pentru bicicletele detectate
            for obj in analysis.objects:
                if obj.object_property == "bicycle":
                    rect = obj.rectangle
                    # Desenăm un dreptunghi roșu
                    draw.rectangle(
                        [(rect.x, rect.y), (rect.x + rect.w, rect.y + rect.h)],
                        outline="red",
                        width=3
                    )
            
            # Salvăm imaginea procesată
            output_path = os.path.join(output_folder, img_name)
            img.save(output_path)


# Example usage
image_folder = "images/bikes"
output_folder = "images/output"
os.makedirs(output_folder, exist_ok=True)
detect_and_draw_bicycles(image_folder, output_folder)

Functiile pentru extragerea de bounding box din imaginile generate 

In [8]:
def extract_bounding_box_from_image(image_path):
    # Citim imaginea
    image = cv2.imread(image_path)
    if image is None:
        return None

    # Convertim imaginea în spațiul de culoare HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definim intervalul pentru culoarea roșie
    lower_red1 = np.array([0, 120, 70])  # Interval pentru roșu deschis
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])  # Interval pentru roșu închis
    upper_red2 = np.array([180, 255, 255])

    # Creăm o mască pentru culoarea roșie
    mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = mask1 | mask2

    # Detectăm contururile
    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Găsim bounding box-ul pentru cel mai mare contur (dacă există)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        return {"x": x, "y": y, "w": w, "h": h}
    else:
        return None
    
def extract_bound_from_all_images(image_folder):
    bounding_boxes = {}
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        bbox = extract_bounding_box_from_image(img_path)
        if bbox:
            bounding_boxes[img_name] = bbox
    return bounding_boxes

Functii pentru calcularea calitatii recunoasterii

In [9]:

def calculate_iou(boxA, boxB):
    # Calculăm coordonatele intersecției
    xA = max(boxA["x"], boxB["x"])
    yA = max(boxA["y"], boxB["y"])
    xB = min(boxA["x"] + boxA["w"], boxB["x"] + boxB["w"])
    yB = min(boxA["y"] + boxA["h"], boxB["y"] + boxB["h"])
    
    # Calculăm aria intersecției
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Calculăm aria uniunii
    boxAArea = boxA["w"] * boxA["h"]
    boxBArea = boxB["w"] * boxB["h"]
    unionArea = boxAArea + boxBArea - interArea
    #print(f"Box A: {boxA}, Box B: {boxB}, InterArea: {interArea}, UnionArea: {unionArea}")
    
    # Calculăm IoU
    iou = interArea / unionArea if unionArea > 0 else 0
    return iou

#Evaluăm performanța algoritmului
def evaluate_detection(predicted_folder, ground_truth_folder):
    predicted_boxes = extract_bound_from_all_images(predicted_folder)
    ground_truth_boxes = extract_bound_from_all_images(ground_truth_folder)
    
    iou_scores = []
    
    
    for img in predicted_boxes.keys():
        if img not in ground_truth_boxes:
            print(f"Image {img} is not in the ground truth data.")
        else:
            pred_box = predicted_boxes[img]
            gt_box = ground_truth_boxes[img]
            
            iou = calculate_iou(pred_box, gt_box)
            print(f"Pentru imaginea {img}, IoU este: {iou:.2f}")
            if predicted_labels[img] == "Bicycle":
                iou_scores.append(iou)
    
    # Calculăm IoU mediu
    mean_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0
    
    # Calculăm precizia medie (mAP)
    correct_predictions = sum(1 for iou in iou_scores if iou > 0.5)
    total_predictions = len(iou_scores)
    precision = correct_predictions / total_predictions if total_predictions > 0 else 0
    
    return mean_iou, precision, iou_scores


Verificarea calitatii recunoasterii biciletor in imagini

In [10]:
# Folderele cu bounding box-uri
predicted_folder = "images/output"  # Bounding box-uri generate de algoritm
ground_truth_folder = "images/myOutput"  # Bounding box-uri corecte

# Calculăm metricile
mean_iou, precision, iou_scores = evaluate_detection(predicted_folder, ground_truth_folder)

# Afișăm rezultatele
print(f"Mean IoU: {mean_iou:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Iou scores:", iou_scores)

Pentru imaginea bike02.jpg, IoU este: 0.90
Pentru imaginea bike03.jpg, IoU este: 0.76
Pentru imaginea bike04.jpg, IoU este: 0.99
Pentru imaginea bike05.jpg, IoU este: 0.87
Pentru imaginea bike06.jpg, IoU este: 0.00
Pentru imaginea bike07.jpg, IoU este: 0.88
Pentru imaginea bike08.jpg, IoU este: 0.00
Pentru imaginea bike09.jpg, IoU este: 0.92
Pentru imaginea bike1.jpg, IoU este: 0.96
Pentru imaginea bike10.jpg, IoU este: 0.89
Pentru imaginea traffic01.jpg, IoU este: 0.00
Pentru imaginea traffic02.jpg, IoU este: 0.00
Pentru imaginea traffic03.jpg, IoU este: 0.00
Pentru imaginea traffic04.jpg, IoU este: 0.00
Pentru imaginea traffic05.jpg, IoU este: 0.00
Pentru imaginea traffic07.jpg, IoU este: 0.00
Image traffic08.jpg is not in the ground truth data.
Pentru imaginea traffic10.jpg, IoU este: 0.00
Mean IoU: 0.72
Precision: 0.80
Iou scores: [0.9006727693834785, 0.7593786227683746, 0.9903846153846154, 0.871438177691855, 0.0, 0.8828151574937071, 0.0022101326079564773, 0.9231451394082182, 0.960